In [1]:
from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.readers.file import PyMuPDFReader
import os
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
from llama_index.core import Document
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, StorageContext
loader = PyMuPDFReader()
docs0 = loader.load(file_path=Path("3.pdf"))
doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [2]:
node_parser = HierarchicalNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(docs)
leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [3]:
embed_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
storage_context = StorageContext.from_defaults(docstore=docstore)

/home/emumba/Documents/ADVANCE RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/emumba/Documents/ADVANCE RAG/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
from llama_index.core import Settings
Settings.llm = Ollama(model="llama3", request_timeout=120.0)
llm=Ollama(model="llama3", request_timeout=200.0)

In [5]:
base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
    embed_model=embed_model 
)

In [6]:
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)
query_engine = RetrieverQueryEngine.from_args(retriever)

In [9]:
query_str = (
    "What is global warming"
    "Describe its effects"
)
response = query_engine.query(query_str)

> Filling in node. Node id: 1f487a9e-43e5-4518-ae26-422f554517a6> Node text: The extra water vapour 
content in the atmosphere will fall again as extra rain 
hence causing fl...

> Merging 4 nodes into parent node.
> Parent node id: 293d478c-f263-45b6-8a2a-6eb144fe12a8.
> Parent node text: V. Global Warming: The Effects 
Predicting the consequences of global warming is 
one of the most...



In [10]:
print(str(response))

Global warming refers to the human-enhanced greenhouse effect that leads to an increase in the average temperature of the Earth's surface. This rise in temperature causes several negative effects, including floods in various regions due to extra water vapor present in the atmosphere falling again as rain. Additionally, evaporation processes from land and sea rise when the weather becomes warmer, leading to droughts in areas where increased precipitation does not compensate for the increased evaporation. This can result in crop failure and famine, particularly in areas with already high temperatures. The extra water vapor content in the atmosphere also falls again as rain, causing further floods. Rising sea levels are another deadly effect of global warming, caused by the rapid melting of ice and glaciers, which can lead to devastation in the form of floods. Furthermore, global warming is projected to cause more heat waves, violent rainfall, and amplification in the severity of hailstor